# Pre-processing

## Importación de bibliotecas

In [1]:
import time
start_time = time.time()
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import io
import requests
import random
import dataframe_image as dfi

## Importación de dataset

In [2]:
nocols = ['Unnamed: 0', 'Unnamed: 0.1']

raw_df = pd.read_csv("./data/cabaventa.csv").drop(columns = nocols)
raw_df.shape

(183810, 25)

## Características del dataset

Se definen funciones accesorias para el recupero de las características del dataset

In [3]:
def rango(col): 
    col = col.dropna()
    if len(col)==0:
        return ""
    if np.issubdtype(col.dtype, np.number):
        return "["+"{:.2f}".format(col.min())+","+"{:.2f}".format(col.max())+"]"
    else: 
        return ""

def ejemplo(col, col_width=30):
    uniques = col.dropna().unique()
    if len(uniques) == 0: 
        return ""
    if len(uniques)<4: 
        s = ",".join([str(x) for x in uniques])
        if len(s)>col_width:
            return s[:col_width]+"..."
        else:
            return s
    else:
        vcounts = col.value_counts().index
        s = ", ".join([str(x) for x in vcounts[:3]])+"..."
        if len(s)>col_width:
            return s[:col_width]+"..."
        else:
            return s
    

def get_dataframe_info(df):
    
    dtypes = df.dtypes    
    nulls = (df.isna().sum()/len(df)).apply(lambda x: "{:.2%}".format(x))
    nuniques = df.nunique()
    ejemplos = df.apply(lambda col: ejemplo(col), axis=0)
    rangos = df.apply(lambda col: rango(col), axis=0)
    info_df = pd.concat([dtypes, nulls, nuniques, ejemplos, rangos], axis=1)
    info_df.columns = ['TipoDato',"PorcentajeNulos","ValoresUnicos","Ejemplos", "Rangos"]
    return info_df

tabla = get_dataframe_info(raw_df)
#tabla.to_latex()
dfi.export(tabla, "img/tabla1.png")
tabla

,TipoDato,PorcentajeNulos,ValoresUnicos,Ejemplos,Rangos
id,object,0.00%,183810,"m8Qw6REGK74j9zZd+GRkVw==, 5Uq8...",
ad_type,object,0.00%,1,Propiedad,
start_date,object,0.00%,362,"2020-10-02, 2020-12-03, 2020-0...",
end_date,object,0.00%,449,"9999-12-31, 2021-06-05, 2021-0...",
created_on,object,0.00%,362,"2020-10-02, 2020-12-03, 2020-0...",
lat,float64,6.23%,69507,"-34.612753999999995, -34.55894...",
lon,float64,6.23%,70182,"-58.41326479999999, -58.475767...",
l1,object,0.00%,1,Argentina,
l2,object,0.00%,1,Capital Federal,
l3,object,1.00%,57,"Palermo, Belgrano, Caballito.....",


## Limpieza Texto y Columnas

### Limpieza inicial

Se eliminan los registros con currency = ARS y se dropea esa columna por carecer de valor predictivo

In [4]:
raw_df = raw_df[raw_df['currency']== "USD"]
raw_df.drop(columns='currency', axis=1, inplace=True) 

El dataset posee 8 registros de tipo de propiedad "Casa de campo". Se considera que con un número tan bajo de registros no será posible realizar predicciones de calidad para ese tipo de propiedad, por lo que se eliminan los registros del dataset.

In [5]:
raw_df = raw_df[raw_df['property_type']!= "Casa de campo"]

### Normalización de campos

En esta parte se realiza una limpieza básica con `regex` de algunas columnas a los fines de normalizar un poco el texto: lowering, punctuation, spaces, digits. La función armada permite customizar la limpieza, por ende es posible no aplicar la misma limpieza para todos los casos. La función genera columnas nuevas con el nombre`f"{col}_cleaned"`  

Dado que las columnas textuales `title` y `description` no fueron utilizadas en la predicción ni analizadas mediante técnicas de embeddings o BOW, no se trataron los NaN en esas columnas. 

In [6]:
from cleaning import CleaningData
#instanciamos Cleaning Data con df

cleaned = CleaningData(data=raw_df)

In [7]:
#Quitamos columnas 
cols_to_drop = [#'Unnamed: 0.1', 'Unnamed: 0', #columnas que vinieron mal en la bajada
                'operation_type', #todos son 'venta'
                'l1', #todas las filas iguales 
                'l2', #todas las filas iguales 
                'ad_type', #todas las filas iguales 
                'l5', #columna con todas las filas nulas
                'l6', #columna con todas las filas nulas
                'created_on', #la columna created_on es igual a la columna start_date
                'price_period' #la columna contiene muchos NaN y un único valor
               ]
data = cleaned.drop_columns(columns=cols_to_drop)
data.shape

Cleaning columns with no valuable information...



(179798, 16)

In [ ]:
#Limpieza title
params = {'lowering':True,'punctuation':True,'accents': True,'strip_spaces':True,'digits':False,'within_spaces':True}
cleaned.text_col_cleaning(text_column="title", params=params)
data.shape

In [ ]:
#Limpieza description
params = {'lowering':True,'punctuation':True,'accents': True,'strip_spaces':True,'digits':False,'within_spaces':True}
cleaned.text_col_cleaning(text_column="description", params=params)
data.shape

In [ ]:
#Limpieza texto de l3
params = {'lowering':False,'punctuation':True,'accents': True,'strip_spaces':True,'digits':False,'within_spaces':True}
cleaned.text_col_cleaning(text_column="l3", params=params)
data.shape

### Mejoras en la delimitación e imputación de barrios

#### Valores nulos

In [ ]:
(data[['lat','lon','l3_cleaned']].isna().sum()/len(data)).apply(lambda x: "{:.2%}".format(x))

Se detectó una mayor presencia de valores nulos en los datos de georreferenciación que en el dato del barrio. Por ende se realizó un trabajo de imputación de nulos partiendo de considerar a la columna `l3_cleaned` con una mayor validez que la columnas `lat` y `lon`. Por ende, fue tomada como "eje" para realizar imputación de nulos. 

#### Errores

* Se analizaron los barrios cuya denomiación no es la denominación oficial: Ejemplo Centro / Microcentro, Las Cañitas, Pompeya, Abasto, Once, etc. 

In [ ]:
data.l3_cleaned.dropna().unique()

* Se analizaron registros para los que la variable `lat` presentó valores fuera del polígono de CABA

In [ ]:
data.lat.describe()

* Se analizaron registros para los que la variable `lon` presentó valores fuera del polígono de CABA

In [ ]:
data.lon.describe()

#### Procesamiento realizado

Se establecieron los siguientes pasos: 

1. A los registros cuya localización no correspondía a CABA se les imputó NaN, para ello se utilizó geoJson del polígono de CABA

In [ ]:
import json
path = "./metadata/provincia.json"
with open(path, "r", encoding="utf-8") as js: 
    js = json.load(js)


poligono_caba = [tuple(x) for x in js['features'][0]['geometry']['coordinates'][0][0]]

from geoBarrios import point_in_polygon

def is_caba(coord): 
    return(point_in_polygon(coord, poligono_caba))

data['is_caba'] = [is_caba((lon, lat)) for lon, lat in zip(data.lon, data.lat)]

print(f"Fueron {(data.is_caba==False).sum()} los casos en que la localización era externa a CABA")

data.loc[~data.is_caba, ['lon','lat']] = np.nan

fig, axs = plt.subplots(ncols=2, figsize=(8,4))

lon , lat = raw_df.lon , raw_df.lat #guardo la localización como vino del dataset
axs[0].scatter(lon,lat)
axs[0].set_title("Localización con Errores")

axs[1].scatter(data.lon,data.lat)
plt.title("Localización sin Errores")
plt.tight_layout()
plt.show()

2. 1: Normalización datos de barrios porteños: dado que eran pocos casos en los que el inmueble pertenecía a un barrio no oficial de CABA según la columna `l3_cleaned` se realizó una imputación manual. Ej: Las Canitas ---> Palermo. Pompeya --> Nueva Pompeya. Abasto --> Almagro. En un trabajo posterior debería ser revisado y refinado ya que hay casos como Once, Parque Centenario, que son zonas pertenecientes a una mezcla de barrios. 

In [ ]:
normalizador = {
    'Pompeya': 'Nueva Pompeya',
    'Las Canitas': 'Palermo',
    'Parque Centenario': 'Caballito', #Puede haber propiedades que en realidad pertenezcan a Almagro o Caballito
    'Centro Microcentro':'San Nicolas',
    'Tribunales':'San Nicolas',
    'Once':'Balvanera',  #puede haber propiedades que en realidd pertenezcan a Almagro.
    'Abasto':"Balvanera",  #puede haber propiedades que en realidad pertenezcan a Almagro. 
    "Catalinas":"Retiro", # la mayor parte de los casos caen ahí (algunos en Boca) 
    'Congreso':"Balvanera", #La mayor parte de los casos caen ahí (algunos en Belgrano)
    "Villa Riachuelo":"Villa Lugano", #Al ser pocas propiedades se imputó el barrio más próximo.
}

data['l3_norm'] = data.l3_cleaned.apply(lambda x: normalizador[x] if x in normalizador else x)
data['l4_nuevo'] = np.where(data.l4.isna()==False, data.l4, 
                            np.where(data.l3_cleaned != data.l3_norm, data.l3_cleaned, data.l3_norm)
                           )


gr1 = data.groupby("l3_norm")['id'].count().sort_values(ascending=False)

gr2 = data.groupby("l4_nuevo")['id'].count().sort_values(ascending=False)

fig, axs = plt.subplots(figsize = (12,10) , ncols=2)


barrios = gr1.index
cantidad1 = gr1.values

sub_barrios = gr2.index
cantidad2 = gr2.values

axs[0].barh(barrios, cantidad1, align='center')
axs[0].invert_yaxis()  # labels read top-to-bottom
axs[0].set_xlabel('Cantidad')
axs[0].set_title('Barrios')

axs[1].barh(sub_barrios, cantidad2, align='center')
axs[1].invert_yaxis()  # labels read top-to-bottom
axs[1].set_xlabel('Cantidad')
axs[1].set_title('Sub-barrios')

plt.tight_layout()
plt.show()

2. 2: Si para un registro el par de columnas `lat`, `lon` presentaron valores NaN, se imputó el valor para cada columna con el promedio de las coordenadas del barrio. Si para dicho registro el dato del barrio fue nulo entonces se quitó toda la fila.

In [ ]:
no_location = (data['lat'].isna()) & data['lon'].isna() & (data['l3_norm'].isna())
print(f"Fueron {no_location.sum()} los casos en que no habia locación")
data = data[~no_location].reset_index(drop=True)

Para aquellas propiedades que sí tenían barrio, pero no las coordenadas geográficas, se imputó para cada columna con el promedio barrial de las coordenadas geográficas presentes en el dataset.

In [ ]:
lat_lon_barrios = data.groupby('l4_nuevo')[['lat', 'lon']].mean().reset_index()
lat_lon_barrios.columns = ['l4_nuevo','lat_barrio', 'lon_barrio']

data = pd.merge(data, lat_lon_barrios, how="left", left_on='l4_nuevo', right_on = 'l4_nuevo')

data['lat'] = np.where(data['lat'].isna(), data['lat_barrio'], data['lat'])
data['lon'] = np.where(data['lon'].isna(), data['lon_barrio'], data['lon'])

3. Luego de este procesamiento si el barrio del registro presentaba un valor NaN se imputó el barrio correspondiente a la coordenada.

In [ ]:
from geoBarrios import geoBarrios, coord_to_nbhd
barrios_dict = geoBarrios()
print(f"Un total de {data.l4_nuevo.isna().sum()} casos poseían barrio nulo y se les imputo el barrio correspondiente a la coordenada") 
data.loc[data.l4_nuevo.isna(), "l4_nuevo"] = data[data.l4_nuevo.isna()].apply(lambda row: coord_to_nbhd(coord=(row.lon,row.lat),polygons_dict=barrios_dict), axis=1).values
data.loc[data.l3_norm.isna(), "l3_norm"] = data[data.l3_norm.isna()].apply(lambda row: coord_to_nbhd(coord=(row.lon,row.lat),polygons_dict=barrios_dict), axis=1).values

In [ ]:
data.drop(columns = ["l3","l3_cleaned","l4","is_caba","lat_barrio","lon_barrio"], inplace=True)

In [ ]:
data.isna().sum()

**Pendiente para un próximo abordaje**

* Para los casos en que la localización era externa a CABA y el barrio no nulo, se imputó un punto centroide del barrio. Eso pudo conducir a errores dado que implícitamente se "confía" en que el dato del barrio fue correcto. Una alternativa sería probar adicionalmente si de la columna `description_cleaned` fuera posible confirmar que no se tratase de casos que efectivamente pertenecieran a localizaciones externas a CABA. 
* Si el barrio y la localización no son nulos pero sí incongruentes (localización pertence a otro barrio) se procedío a imputar el punto centroide del barrio. . 

In [ ]:
data.shape

## Trabajo sobre las variables: superficie total, superficie cubierta, ambientes, dormitorios y precio

#### Exploración de valores nulos

In [ ]:
cols = ["surface_total","surface_covered","rooms","bedrooms", "price"]
(data[cols].isna().sum()/len(data)).apply(lambda x: "{:.2%}".format(x))

Se exploraron los valores no nulos las columnas `rooms` y `bedrooms`, según el tipo de propiedad. 

In [ ]:
100*(data[['rooms','bedrooms']].isnull()==False).groupby(data['property_type']).sum().div(data.groupby("property_type")['rooms','bedrooms'].apply(lambda gr: len(gr)), axis=0) 

* Las columnas `rooms` y `bedrooms` poseen muchos nulos siempre que los avisos se traten de cocheras, depósitos, locales comerciales, lotes, oficinas, otros. Para las propiedades utilizadas como vivienda en cambio la presencia de nulos es mucho menor. Por ende en los casos en que las propiedades no sean para vivienda los nulos se completan con valor 0. 
* Luego de esto dada la alta correlación entre `surface_total` y `surface_covered` y entre `rooms` y `bedrooms`, para los casos en que el par tuviera datos nulos se procedió a eliminar la fila ante la dificultad para imputar nulos en esos casos. 
* Para la variable `price` se eliminaron las filas que tenían nulos ya que por ser la variable sobre la cual se constuirá la variable respuesta se decidió no realizar imputación de valores faltantes sobre la misma. 

#### Exploración de variables

* `surface_total < surface_covered`

In [ ]:
data.surface_total.describe()

In [ ]:
(data.surface_total < data.surface_covered).sum() 

* outliers en `surface_covered`

In [ ]:
data.surface_covered.describe()

* `rooms < bedrooms`

In [ ]:
(data.rooms < data.bedrooms).sum() 

* outliers en `rooms`

In [ ]:
data.rooms.describe()

* outliers en `bedrooms`

In [ ]:
data.bedrooms.describe()

#### Procesamiento realizado

0) Imputar 0 en las columnas `['rooms','bedrooms']` cuando se trate de cocheras, depósitos, locales comerciales, lotes, oficinas, otros. 
1) Borrar filas donde los pares de columnas `['rooms','bedrooms']` o `['surface_total','surface_covered']` contengan nulos. 
2) Intercambiar los valores de las columnas cuando `rooms < bedrooms` o `surface_total < surface_covered`. 
3) Eliminar filas donde `price` sea nulo. 
4) Si para una fila uno de los valores de `['surface_total', 'surface_covered']` contiene valor outlier y el otro no es outlier, se le imputó un `NaN` al dato outlier y se dejó dicho valor para ser imputado luego por método MICE. En caso que los dos valores fueran outliers o uno outlier y el otro `NaN` se borró la fila. 
5) Si para una fila uno de los valores de `['rooms', bedrooms']` contiene valor outlier y el otro no es outlier, se le imputó un `NaN` al dato outlier y se dejó dicho valor para ser imputado luego por método MICE. En caso que los dos valores fueran outliers o uno outlier y el otro `NaN` se borró la fila. 
6) Se borraron los valores outliers de `price`
7) Luego de estos pasos se procedió a imputar los valores nulos con el método multivariado MICE en las columnas `['rooms', bedrooms','surface_total','surface_covered']` utilizando también la variable `price` como predictor. 

Reemplazar rooms y bedrooms 0 para property_type 'Local comercial', 'Depósito', 'Cochera', 'Lote', 'Oficina', 'Otro'

In [ ]:
data.loc[data.property_type.isin(["Cochera","Depósito","Local comercial","Lote","Oficina","Otro"]),["rooms","bedrooms"]] = data.loc[data.property_type.isin(["Cochera","Depósito","Local comercial","Lote","Oficina","Otro"]),["rooms","bedrooms"]].fillna(0)

In [ ]:
data.shape

In [ ]:
nas_rb = data[data[['rooms', 'bedrooms']].isna().any(axis=1)][['rooms', 'bedrooms', 'l3_norm']].groupby('l3_norm').count()

In [ ]:
import missingno as msno

msno.matrix(data)

In [ ]:
#punto 1
data.dropna(subset = ['rooms', 'bedrooms'], how = 'all', inplace = True)
data.shape


In [ ]:
data.dropna(subset = ['surface_total', 'surface_covered'], how = 'all', inplace=True)
data.shape

In [ ]:
data.dropna(subset =['price'], inplace=True)
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
data[['rooms','bedrooms']].isnull().groupby(data['property_type']).sum()

* outlieres en `surface_total`

In [ ]:
#punto 2
idx = data[data.surface_total<data.surface_covered].index
data.loc[idx,['surface_total','surface_covered']] = data.loc[idx,['surface_covered','surface_total']].values

idx = data[data.rooms<data.bedrooms].index
data.loc[idx,['rooms','bedrooms']] = data.loc[idx,['bedrooms','rooms']].values

4) Si para una fila uno de los valores de ['surface_total', 'surface_covered'] contiene valor outlier y el otro no es outlier, se le imputó un NaN al dato outlier y se dejó dicho valor para ser imputado luego por método MICE.

In [ ]:
#Generamos quantiles .01 y .99 para surface_total
data_st = data[['l3_norm', 'property_type', 'surface_total', 'rooms']]
data_st_q99 = data_st.groupby(['l3_norm', 'property_type', 'rooms']).quantile(.99)
data_st_q99.rename(columns = {'surface_total':'p99_surface_total'}, inplace = True)

data_sc = data[['l3_norm', 'property_type', 'surface_covered', 'rooms']]
data_sc_q99 = data_sc.groupby(['l3_norm', 'property_type', 'rooms']).quantile(.99)
data_sc_q99.rename(columns = {'surface_covered':'p99_surface_covered'}, inplace = True)

In [ ]:
data = data.merge(data_st_q99, how = "left", on = (['l3_norm', 'property_type', 'rooms']))
data = data.merge(data_sc_q99, how = "left", on = (['l3_norm', 'property_type', 'rooms']))

In [ ]:
##Generamos quantiles .01 y .99 para surface_covered
data_st_q01 = data_st.groupby(['l3_norm', 'property_type', 'rooms']).quantile(.01)
data_st_q01.rename(columns = {'surface_total':'p01_surface_total'}, inplace = True)

data_sc_q01 = data_sc.groupby(['l3_norm', 'property_type', 'rooms']).quantile(.01)
data_sc_q01.rename(columns = {'surface_covered':'p01_surface_covered'}, inplace = True)

In [ ]:
data = data.merge(data_st_q01, how = "left", on = (['l3_norm', 'property_type', 'rooms']))
data = data.merge(data_sc_q01, how = "left", on = (['l3_norm', 'property_type', 'rooms']))

In [ ]:
# Registros con un outlier en surface_total
data[(data['surface_total'] > data['p99_surface_total']) | (data['surface_total'] < data['p01_surface_total'])]

In [ ]:
# Registros con un outlier en surface_covered
data[(data['surface_covered'] > data['p99_surface_covered']) | (data['surface_covered'] < data['p01_surface_covered'])]

In [ ]:
#Elimino registros con 2 precios outliers
data = data[(data['surface_total'] < data['p99_surface_total']) & (data['surface_total'] > data['p01_surface_total']) & (data['surface_covered'] < data['p99_surface_covered']) & (data['surface_covered'] > data['p01_surface_covered'])]

In [ ]:
# Registros con un outlier en surface_covered
data[(data['surface_covered'] > data['p99_surface_covered']) | (data['surface_covered'] < data['p01_surface_covered'])]

In [ ]:
# Registros con un outlier en surface_total
data[(data['surface_total'] > data['p99_surface_total']) | (data['surface_total'] < data['p01_surface_total'])]

In [ ]:
#Paso a NA el registro outlier de surface_total o surface_covered
data['surface_total'] = np.where((data['surface_total'] > data['p99_surface_total']) | (data['surface_total'] < data['p01_surface_total']), np.nan, data['surface_total'])
data['surface_covered'] = np.where((data['surface_covered'] > data['p99_surface_covered']) | (data['surface_covered'] < data['p01_surface_covered']), np.nan, data['surface_covered'])
#Dropeo columnas accesorias
data = data.drop(columns= ['p99_surface_total', 'p01_surface_total', 'p99_surface_covered', 'p01_surface_covered'], axis = 1)

In [ ]:
data.isnull().sum()

In [ ]:
data.shape

En caso que los dos valores de 'surface_covered' y 'surface_total' fueran outliers o uno outlier y el otro NaN se borró la fila

In [ ]:
data = data.dropna(subset=['surface_covered', 'surface_total'], how='all')

In [ ]:
data.shape

 5) Si para una fila uno de los valores de ['rooms', bedrooms'] contiene valor outlier y el otro no es outlier, se le imputó un NaN al dato outlier y se dejó dicho valor para ser imputado luego por método MICE. En caso que los dos valores fueran outliers o uno outlier y el otro NaN se borró la fila.

In [ ]:
data.groupby(['property_type','rooms']).size()

In [ ]:
data.groupby(['property_type','bedrooms']).size()

Se imputan con 1 los registros con valor negativo de bedrooms

In [ ]:
data['bedrooms'] = np.where(data['bedrooms']<0, 1, data['bedrooms'])

In [ ]:
data.groupby(['property_type','bedrooms']).size()

 6) Se borraron los valores outliers de price agrupados por barrio, tipo de propiedad y cuartos

In [ ]:
data.shape

In [ ]:
data_pr = data[['l3_norm', 'property_type', 'price', 'rooms']]
data_pr_q99 = data_pr.groupby(['l3_norm', 'property_type', 'rooms']).quantile(.99)
data_pr_q99.rename(columns = {'price':'p99_price'}, inplace = True)
data_pr_q01 = data_pr.groupby(['l3_norm', 'property_type', 'rooms']).quantile(.01)
data_pr_q01.rename(columns = {'price':'p01_price'}, inplace = True)
data = data.merge(data_pr_q99, how = "left", on = (['l3_norm', 'property_type', 'rooms']))
data = data.merge(data_pr_q01, how = "left", on = (['l3_norm', 'property_type', 'rooms']))
#Eliminamos precios outliers
data = data[(data['price']< data['p99_price']) & (data['price'] > data['p01_price'])]

7. Reimputación de bathrooms

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
df = data 
index_orig= df.index
index_orig2= df.index.name

In [ ]:
index_orig

In [ ]:
# Veo cuales son los valores medios de 'bathroom" para cada cantidad de ambientes por cada tipo de propiedad --> y lo guardo en un nuevo df llamado 'bath_med'
cols = ['rooms','bathrooms', 'property_type']
medxprop = df[cols].groupby(['rooms','property_type']).median()#.rename({'bathrooms':'bathrooms_med'},axis=1).round(0)
medxprop = medxprop.rename({'bathrooms':'bathrooms_med'},axis=1).round(0)
bath_med = medxprop.reset_index()
bath_med

In [ ]:
# agrego una nueva columna al df con esa mediana calculada para luego imputar los nulls en bathrooms con ese dato
df = df.merge(bath_med, 'left', on = ['rooms','property_type'])

In [ ]:
# imputamos: le asignamos la mediana x tipo y cant. rooms
mask = df['bathrooms'].isna()
df.loc[mask, 'bathrooms'] = df.loc[mask,'bathrooms_med']

In [ ]:
# Verificamos que no hay nulls
df['bathrooms'].isna().value_counts()

In [ ]:
# le volvemos a poner el indice y lo renombramos al nombre previo:
df = df.set_index(index_orig)

In [ ]:
# ACCION REQUERIDA: volver al nombre original --> df_Actual_Name
data = df #--> CAMBIAR df_Actual_Name

## Resultados de Pre-procesamiento

### Columnas

Columnas iniciales: 

In [ ]:
raw_df.columns

Columnas a exportar por este script:

In [ ]:
data.columns

### Filas

#### Imputación de datos faltantes

El dataset presentaba una gran cantidad de datos faltantes 

In [ ]:
raw_df.isnull().sum()

Más importante aún, todos los registros del dataset poseían al menos un dato faltante, por lo que no existían registros aptos para utilizar en un algoritmo predictivo.

In [ ]:
len(raw_df.isnull().any(axis=1)) #Nro de filas con al menos un dato faltante

In [ ]:
len(raw_df.isnull().any(axis=1)) - len(~raw_df.isnull().any(axis=1)) #Nro de filas del dataset - Nro de filas con al menos un dato faltante

El proceso de normalización, imputación y selección inicial de features permitió imputar y recuperar el siguiente número de filas 

In [ ]:
len(data)

#### Registros iniciales y finales

Filas iniciales:

In [ ]:
len(raw_df)

Filas a exportar por este script:

In [ ]:
len(data)

## Exportación

Este csv es recuperado por el script de Feature Engineering para generar variables adicionales 

In [ ]:
data.to_csv('resultados/cabaventa_preproc.csv')

In [ ]:
import datetime
end_time = time.time()
elapsed = str(datetime.timedelta(seconds=end_time - start_time))
print(f"Time execution is {elapsed}")